In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import numpy as np 
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import random

sns.set_context('notebook',font_scale=1.5)


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Mercari/p_train.csv')

In [ ]:
data.head()

,Unnamed: 0,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_main,category_sub1,category_sub2
0,0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,uknown,10.0,1,No description yet,Men,Tops,T-shirts
1,1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces


In [ ]:
data = data.iloc[: , 1:]

In [ ]:
data.isnull().sum()

train_id             0
name                 0
item_condition_id    0
category_name        0
brand_name           0
price                0
shipping             0
item_description     0
category_main        0
category_sub1        0
category_sub2        0
dtype: int64

In [ ]:
data = data.drop('category_name', axis=1)
data.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_main,category_sub1,category_sub2
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces


In [ ]:
data['item_condition_id'] = data['item_condition_id'].astype('category')
data['shipping'] = data['shipping'].astype('category')

In [ ]:
data.dtypes

train_id                int64
name                   object
item_condition_id    category
brand_name             object
price                 float64
shipping             category
item_description       object
category_main          object
category_sub1          object
category_sub2          object
dtype: object

#### Frequency encoding

#### Brand Name

In [ ]:
temp = data['brand_name'].value_counts()
temp.head()

uknown               8553
PINK                  730
Nike                  699
Victoria's Secret     627
LuLaRoe               405
Name: brand_name, dtype: int64

In [ ]:
data['brand_name_count'] = data['brand_name'].apply(lambda x: temp[x])
data[['brand_name', 'brand_name_count']].head()

,brand_name,brand_name_count
0,uknown,8553
1,Razer,3
2,Target,22
3,uknown,8553
4,uknown,8553


#### target encoding

In [ ]:
brand_name_mean_encoding = data.groupby(['brand_name']).agg({'price':['mean']}).reset_index()

data = data.merge(brand_name_mean_encoding,on='brand_name',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"(price, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180


#### item condition id

#### target encoding

In [ ]:
item_condition_id_mean_encoding = data.groupby(['item_condition_id']).agg({'price':['mean']}).reset_index()

data = data.merge(item_condition_id_mean_encoding,on='item_condition_id',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","(price_y, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259


#### shipping

In [ ]:
temp3 = data['shipping'].value_counts()
temp3.head()

0    11008
1     8948
Name: shipping, dtype: int64

In [ ]:
data['shipping_count'] = data['shipping'].apply(lambda x: temp3[x])
data[['shipping', 'shipping_count']].head()

,shipping,shipping_count
0,1,8948
1,0,11008
2,1,8948
3,1,8948
4,0,11008


#### target encoding

In [ ]:
shipping_mean_encoding = data.groupby(['shipping']).agg({'price_x':['mean']}).reset_index()

data = data.merge(shipping_mean_encoding,on='shipping',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","(price_y, mean)",shipping_count,"(price_x, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650,8948,21.731001
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650,11008,29.821403
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259,8948,21.731001
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259,8948,21.731001
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259,11008,29.821403


#### category main

In [ ]:
temp4 = data['category_main'].value_counts()
temp4.head()

Women          8895
Beauty         2968
Kids           2272
Electronics    1610
Men            1258
Name: category_main, dtype: int64

In [ ]:
data['category_main_count'] = data['category_main'].apply(lambda x: temp4[x])
data[['category_main', 'category_main_count']].head()

,category_main,category_main_count
0,Men,1258
1,Electronics,1610
2,Women,8895
3,Home,846
4,Women,8895


#### target encoding

In [ ]:
category_main_mean_encoding = data.groupby(['category_main']).agg({'price_x':['mean']}).reset_index()

data = data.merge(category_main_mean_encoding,on='category_main',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price_x_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","(price_y, mean)",shipping_count,"('price_x', 'mean')_x",category_main_count,"(price_x_y, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650,8948,21.731001,1258,35.191574
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650,11008,29.821403,1610,32.832919
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259,8948,21.731001,8895,28.358572
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259,8948,21.731001,846,24.395981
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259,11008,29.821403,8895,28.358572


#### category sub 1

In [ ]:
temp5 = data['category_sub1'].value_counts()
temp5.head()

Makeup              1802
Athletic Apparel    1726
Shoes               1392
Tops & Blouses      1385
Jewelry              858
Name: category_sub1, dtype: int64

In [ ]:
data['category_sub1_count'] = data['category_sub1'].apply(lambda x: temp5[x])
data[['category_sub1', 'category_sub1_count']].head()

,category_sub1,category_sub1_count
0,Tops,318
1,Computers & Tablets,49
2,Tops & Blouses,1385
3,Home Décor,318
4,Jewelry,858


#### target encodig

In [ ]:
category_sub1_mean_encoding = data.groupby(['category_sub1']).agg({'price_x_x':['mean']}).reset_index()

data = data.merge(category_sub1_mean_encoding,on='category_sub1',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price_x_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","(price_y, mean)",shipping_count,"('price_x', 'mean')_x",category_main_count,"(price_x_y, mean)",category_sub1_count,"(price_x_x, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650,8948,21.731001,1258,35.191574,318,18.921384
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650,11008,29.821403,1610,32.832919,49,71.551020
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259,8948,21.731001,8895,28.358572,1385,17.555235
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259,8948,21.731001,846,24.395981,318,22.235849
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259,11008,29.821403,8895,28.358572,858,25.737762


#### category sub 2

In [ ]:
temp6 = data['category_sub2'].value_counts()
temp6.head()

Other                      860
Pants, Tights, Leggings    806
Face                       713
T-Shirts                   607
Lips                       478
Name: category_sub2, dtype: int64

In [ ]:
data['category_sub2_count'] = data['category_sub2'].apply(lambda x: temp6[x])
data[['category_sub2', 'category_sub2_count']].head()

,category_sub2,category_sub2_count
0,T-shirts,229
1,Components & Parts,13
2,Blouse,279
3,Home Décor Accents,163
4,Necklaces,276


#### target encoding

In [ ]:
category_sub2_mean_encoding = data.groupby(['category_sub2']).agg({'price_x_x':['mean']}).reset_index()

data = data.merge(category_sub2_mean_encoding,on='category_sub2',how='left')

In [ ]:
data.head()

,train_id,name,item_condition_id,brand_name,price_x_x_x,shipping,item_description,category_main,category_sub1,category_sub2,brand_name_count,"('price', 'mean')_x","(price_y, mean)",shipping_count,"('price_x', 'mean')_x",category_main_count,"(price_x_y, mean)",category_sub1_count,"('price_x_x', 'mean')_x",category_sub2_count,"(price_x_x_y, mean)"
0,0,MLB Cincinnati Reds T Shirt Size XL,3,uknown,10.0,1,No description yet,Men,Tops,T-shirts,8553,20.537180,26.092650,8948,21.731001,1258,35.191574,318,18.921384,229,17.471616
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,3,37.000000,26.092650,11008,29.821403,1610,32.832919,49,71.551020,13,50.153846
2,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,22,18.090909,26.136259,8948,21.731001,8895,28.358572,1385,17.555235,279,15.516129
3,3,Leather Horse Statues,1,uknown,35.0,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,8553,20.537180,26.136259,8948,21.731001,846,24.395981,318,22.235849,163,22.429448
4,4,24K GOLD plated rose,1,uknown,44.0,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,8553,20.537180,26.136259,11008,29.821403,8895,28.358572,858,25.737762,276,24.449275


#### If brand yes/no feature

In [ ]:
data.to_csv('p1_train.csv')
!cp p1_train.csv "/content/drive/MyDrive/Mercari/p1_train.csv"